# 4-3 OpenPose のネットワーク構成と実装

## OpenPose を構成するモジュール
以下に OpenPose のモジュール構成を示す．

<img src="../image/p208.png">

OpenPose は7個のモジュールから構成される．
画像の特徴量を抽出する Feature モジュールと，heatmaps と PAFs を出力する Stage1 ~ Stage6 の6個の Stage モジュールを用意する．  
前処理された画像データは最初に Feature モジュールに入力され，128チャネルの特徴量に変換される．
Feature モジュールでは VGG-19 を使用し，出力される画像サイズは 1/8 になる．
そのため，Feature モジュールの出力は 128×46×46 となる．  
Feature モジュールの出力は，その後 Stage1 から Stage6 へと送られる．
Stage1 では Feature モジュールの出力を2つのサブネットワークに入力する．
それぞれのサブネットワークを block1_1 と block1_2 とする．
前者は 38×46×46 サイズの PAFsを，後者は 19×46×46 サイズの heatmaps を出力する．  
簡単に姿勢推定をするだけなら Stage1 の出力を用いればよいが，これだけでは十分な精度が得られないため，Stage1 と Feature モジュールの出力を使ってさらに精度の良い姿勢推定を行う．  
Stage1 と Feature モジュールの出力を全てチャネル方向に結合させ，185×46×46 のテンソルとして Stage2 の block2_1 と block2_2 に入力する．
Stage2 でも同様に PAFs と Heatmaps を出力する．
これによって，Stage1 より精度の高い姿勢推定の結果が得られる．  
同様に前段の Stage モジュールの出力（38×46×46，19×46×46）と Feature モジュールの出力（128×46×46）をチャネル方向に結合したものを，次段の Stage モジュールに入力することを繰り返し，最終的に Stage6 の出力する PAFs と heatmaps を使って姿勢推定を行う．

## OpenPoseNet の実装
これまで通り，コンストラクタで各モジュールを生成し，順伝播関数 forward を定義する．
OpenPose のネットワークの学習を行う際には，各 Stage の PAFs と heatmaps に対して教師データのものとの損失値を計算する．
最終的な forward 関数の出力は Stage6 の PAFs と heatmaps，及び各 Stage モジュールの PAFs と heatmaps を辞書型変数にまとめた saved_for_lossとなる．

In [2]:
import torch
import torch.nn as nn

class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet, self).__init__()
        
        # Feature モジュール
        self.model0 = OpenPose_Feature()
        
        # Stage モジュール
        # PAFs (Part Affinity Fields) 側
        self.model1_1 = make_OpenPose_block("block1_1")
        self.model2_1 = make_OpenPose_block("block2_1")
        self.model3_1 = make_OpenPose_block("block3_1")
        self.model4_1 = make_OpenPose_block("block4_1")
        self.model5_1 = make_OpenPose_block("block5_1")
        self.model6_1 = make_OpenPose_block("block6_1")
        
        # confidence heatmap 側
        self.model1_2= make_OpenPose_block("block1_2")
        self.model2_2= make_OpenPose_block("block2_2")
        self.model3_2= make_OpenPose_block("block3_2")
        self.model4_2= make_OpenPose_block("block4_2")
        self.model5_2= make_OpenPose_block("block5_2")
        self.model6_2= make_OpenPose_block("block6_2")
        
        
    def forward(self, x):
        ''' 順伝播関数の定義 '''
        
        # Feature モジュール
        out1 = self.model0(x)
        
        # Stage1
        out1_1 = self.model1_1(out1) # PAFs
        out1_2 = self.model1_2(out1) # confidence heatmaps
        
        # Stage2
        out2 = torch.cat([out1_1, out1_2, out1], 1) # チャネルの次元で結合
        out2_1 = self.model2_1(out2) # PAFs
        out2_2 = self.model2_2(out2) # confidence heatmaps
        
        # Stage3
        out3 = torch.cat([out2_1, out2_2, out1], 1) # チャネルの次元で結合
        out3_1 = self.model3_1(out3) # PAFs
        out3_2 = self.model3_2(out3) # confidence heatmaps
        
        # Stage4
        out4 = torch.cat([out3_1, out3_2, out1], 1) # チャネルの次元で結合
        out4_1 = self.model4_1(out4) # PAFs
        out4_2 = self.model4_2(out4) # confidence heatmaps
        
        # Stage5
        out5 = torch.cat([out4_1, out4_2, out1], 1) # チャネルの次元で結合
        out5_1 = self.model5_1(out5) # PAFs
        out5_2 = self.model5_2(out5) # confidence heatmaps
        
        # Stage6
        out6 = torch.cat([out5_1, out5_2, out1], 1) # チャネルの次元で結合
        out6_1 = self.model6_1(out6) # PAFs
        out6_2 = self.model6_2(out6) # confidence heatmaps
        
        # 損失の計算用に各 Stage の結果を格納
        saved_for_loss = []
        saved_for_loss.append(out1_1)
        saved_for_loss.append(out1_2)
        saved_for_loss.append(out2_1)
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)
        
        # 最終的な PAFs の out6_1 と confidence heatmap の out6_2，
        # 損失計算用に各ステージでの PAFs と heatmap を格納した saved_for_loss を出力
        # out6_1:torch.Size([minibatch, 38, 46, 46])
        # out6_2:torch.Size([minibatch, 19, 46, 46])
        # saved_for_loss:[out1_1, out_1_2, ・・・, out6_2]
        
        return (out6_1, out6_2), saved_for_loss